# Voorbeeld van omzetting door Veg2Hab

Dit notebook doorloopt alle stappen die tot nu toe geimplementeerd zijn voor Veg2Hab. Het is een voorbeeld van hoe de omzetting van vegetatie naar habitattypekaart kan verlopen.

Eerst worden alle benodigde resources geinstantieerd (was-wordt lijst, definitietabel, fgr-kaart). Hierna wordt de omzetting stap voor stap uitgevoerd.

In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
import pandas as pd
from veg2hab.fgr import FGR

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(3)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


,VvN,SBB
0,14rg10,14-l
1,None,7-a
2,None,8b-b


### Definitietabel opschonen en inladen

VvN- en SBB-codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [3]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_json_def = Path("../data/definitietabel_json.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_json_def, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head(3)

Alle SBB codes zijn valide
Alle VvN codes zijn valide


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


,DT regel,Habitattype,Kwaliteit,SBB,VvN,mits,mozaiek,json,Criteria
0,2,H1110_A,Kwaliteit.GOED,50a,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...",((FGR is Getijdengebied of FGR is Noordzee) en...
1,3,H1110_A,Kwaliteit.GOED,50b,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...",((FGR is Getijdengebied of FGR is Noordzee) en...
2,4,H1110_A,Kwaliteit.GOED,50c,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...",((FGR is Getijdengebied of FGR is Noordzee) en...


### FGR inladen

Bij het inladen worden de FGR typen omgezet naar de FGRType Enum, en hierdoor vind ook validatie plaats.

In [4]:
fgr = FGR(Path("../data/bronbestanden/FGR.json"))
fgr.gdf.head(3)

,fgr,geometry
0,FGRType.DU,"POLYGON ((105147.000 526863.188, 105092.000 52..."
1,FGRType.DU,"POLYGON ((111314.000 552409.000, 111289.008 55..."
2,FGRType.DU,"POLYGON ((119244.333 577690.288, 119230.924 57..."


## Inladen en omzetten van een access kartering

- Voor ieder vlak in de kartering wordt de (complexe) SBB code opgezocht.
- Deze SBB-codes worden opgeschoond en gevalideerd.
- De SBB codes worden toegevoegd aan de dataframe van de kartering als VegTypeInfo instance.
  
De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [5]:
shape_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
csvs_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013/")
shape_elm_id_column = "ElmID"

access_kartering = Kartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

access_kartering.gdf.head(3)


,ElmID,Datum,Projcode,Opp,Dlgebied,Opmerking,VegType,LabelKaart,geometry,intern_id,Locatie,VegTypeInfo
0,28049,19072013,864,0.376174,RM-Kern,None,"09A2-6.e,h,v.90",09A2-6(2),"POLYGON ((189433.818 538314.302, 189439.215 53...",1809,1809,"[(90%, VvN: [], SBB: ['9a2a']), (10%, VvN: [],..."
1,15034,10092013,864,1.044266,RM-Oost,None,50A-1.e,50A-1,"POLYGON ((190568.500 539171.122, 190538.023 53...",815,815,"[(100%, VvN: [], SBB: ['50a'])]"
2,13503,14102013,864,0.217410,RM-Noord,None,"09A2-1.e,h,q,v.90",09A2-1(2),"POLYGON ((189538.104 539409.332, 189532.399 53...",650,650,"[(90%, VvN: [], SBB: ['9a2a']), (10%, VvN: [],..."


#### Opzoeken en toevoegen VvN aan kartering

In [6]:
access_kartering.apply_wwl(wwl)
# Voorbeeld van 3 vegtypeinfos in 1 geometry
access_kartering.gdf["VegTypeInfo"].iloc[3]

[VegTypeInfo(percentage=80, SBB=[SBB(klasse='9', verbond='b', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='a')], VvN=[]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='g')], VvN=[VvN(klasse='9', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='3')]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)], VvN=[VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)])]

#### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [7]:
access_kartering.apply_deftabel(dt)
access_kartering.gdf["HabitatVoorstel"]

0       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1       [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
2       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
3       [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
4       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
                              ...                        
1944    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1945    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1946    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1947    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1948    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
Name: HabitatVoorstel, Length: 1949, dtype: object

In [8]:
# Voorbeeld met voor 2 VegTypeInfo's waarbij de eerste twee mogelijke voorstellen heeft
access_kartering.gdf["HabitatVoorstel"][45]

[[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), vegtypeinfo=VegTypeInfo(percentage=40, SBB=[SBB(klasse='39', verbond='a', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='f')], VvN=[VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None)]), habtype='H2180_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=178, mits=FGRCriterium(fgrtype=<FGRType.DU: 'Duinen'>), mozaiek=None, match_level=6, percentage=40),
  HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassoc

### De mitsen checken en de HabitatVoorstellen omzetten naar HabitatKeuzes

In [9]:
access_kartering.check_mitsen(fgr)
print(access_kartering.gdf["HabitatKeuze"][5][1])
print(access_kartering.gdf["VegTypeInfo"][5][1])

HabitatKeuze(status=<KeuzeStatus.MEERDERE_KLOPPENDE_MITSEN: 4>, opmerking="Er zijn meerdere habitatvoorstellen die aan hun mitsen voldoen; Kloppende mitsen: [['11ba2', 'FGR is Laagveengebied'], ['11ba2a', 'FGR is Laagveengebied'], ['11ba2b', 'FGR is Laagveengebied']]", debug_info='["(10%, VvN: [\'11ba2\', \'11ba2a\', \'11ba2b\'], SBB: [\'11b2\'])", "(10%, VvN: [\'11ba2\', \'11ba2a\', \'11ba2b\'], SBB: [\'11b2\'])", "(10%, VvN: [\'11ba2\', \'11ba2a\', \'11ba2b\'], SBB: [\'11b2\'])"]', habitatvoorstellen=[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='11', orde='b', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='11', orde='b', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), vegtypeinfo=VegTypeInfo(percentage=10, SBB=[SBB(klasse='11', verbond='b', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None)], VvN=[VvN(klasse='11', orde='b'

### De kartering formatten als een Habitattypenkartering

Dit pakt de HabitatVoorstellen uit en format het geheel als een Habitattypenkartering zoals beschreven in Gegevens Leverings Protocol (uitvraag bijlage 3a).

Voor nu checken we nog geen mitsen en mozaiek en wordt voor ieder VegTypeInfo het eerste HabitatVoorstel gebruikt.

In [10]:
final_format = access_kartering.as_final_format()
final_format

,Area,Opm,geometry,Habtype1,Perc1,Opp1,Kwal1,Opm1,VvN1,SBB1,...,Perc5,Opp5,Kwal5,Opm5,VvN5,SBB5,_Status5,_VvNdftbl5,_SBBdftbl5,_VgTypInf5
0,0.376174,None,"POLYGON ((189433.818 538314.302, 189439.215 53...",H7140_B,90,33.8557,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.044266,None,"POLYGON ((190568.500 539171.122, 190538.023 53...","['H2190_A', 'H3110', 'H3130', 'H3140', 'H3150'...",100,"[104.4266, 104.4266, 104.4266, 104.4266, 104.4...","['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ...",Er zijn meerdere habitatvoorstellen die aan hu...,"[None, None, None, None, None, None, None, Non...","['50a', '50a', '50a', '50a', '50a', '50a', '50...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.217410,None,"POLYGON ((189538.104 539409.332, 189532.399 53...",H7140_B,90,19.5669,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.887805,None,"POLYGON ((190801.313 539561.740, 190819.617 53...",H3160,80,151.024,G,Er is een duidelijke keuze. Kloppende mits: ge...,None,9b-a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.495061,None,"POLYGON ((190497.231 537724.945, 190505.062 53...",H0000,58,28.7135,None,,8bb4d,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944,0.092911,None,"POLYGON ((188748.713 538720.165, 188740.937 53...",H91D0,90,8.36199,M,Er is een duidelijke keuze. Kloppende mits: ge...,36aa2a,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1945,0.090002,None,"POLYGON ((188717.598 538767.120, 188713.450 53...",H0000,90,8.10018,None,,32rg3,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1946,0.011078,None,"POLYGON ((189977.881 538379.974, 189975.341 53...",H0000,60,0.66468,None,Er zijn geen habitatvoorstellen waarvan de mit...,8ba2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947,0.274077,None,"POLYGON ((189897.152 537982.231, 189906.500 53...","['H2190_A', 'H3110', 'H3130', 'H3140', 'H3150'...",10,"[2.7407700000000004, 2.7407700000000004, 2.740...","['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ...",Er zijn meerdere habitatvoorstellen die aan hu...,"[None, None, None, None, None, None, None, Non...","['50a', '50a', '50a', '50a', '50a', '50a', '50...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
access_kartering.final_format_to_shp(Path("../testing/veg2hab_output.shp"))